In [12]:
import pandas as pd
from tqdm import tqdm
import datetime
import duckdb

In [ ]:


#Listes des activités retenues par bribes d'APE (voire nomenclature NAF)
activites_annotees = [
("4511Z", "Commerce de voitures et véhicules légers"),
("4519Z", "Commerce d'autres véhicules automobiles"),
("45",    "Entretien et réparation véhicules automobiles"),
("454",   "Commerce et réparation de motocyclettes"),
("461",   "Intermédiaires du commerce de gros"),
("462",   "Commerce de gros produits agricoles et animaux vivants"),
("463",   "Commerce de gros aliments, boissons et tabac"),
("466",   "Commerce de gros autres équipements industriels"),
("467",   "Autres commerces de gros spécialisés"),
("471",   "Commerce de détail en magasin non spécialisé"),
("472",   "Commerce de détail alimentaire en magasin spécialisé"),
("473",   "Commerce de détail de carburants en magasin spécialisé"),
("478",   "Commerce de détail sur éventaires et marchés"),
("551",   "Hôtels et hébergement similaire"),
("552",   "Hébergement touristique et autre hébergement courte durée"),
("5610A", "Restauration traditionnelle"),
("5610B", "Cafétérias et autres libres-services"),
("5610C", "Restauration rapide"),
("563",    "Débits de boissons")
]

activites = [activites_annotees[i][0] for i in range(len(activites_annotees))]

#Départements d'IDF
dep = ["75","77","78","91","92","93","94","95"]

#Date minimale 
date_min = [2010,1,1]


#Récupération des données depuis le cloud ssp cloud S3
dates_select = [1980,1990,2000,2005,2010,2015,2017,2018,2019,2020,2021,2022,2023,2024]

dfs_rne = dict()
df_names = [(f"df_{dates_select[i-1]}_{dates_select[i]}",i) for i in range(1,len(dates_select))]

url_dataRNE = ['https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_1980_1990.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_1990_2000.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2000_2005.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2005_2010.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2010_2015.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2015_2017.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2017_2018.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2018_2019.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2019_2020.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2020_2021.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2021_2022.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2022_2023.parquet',
'https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_RNE/RNE_2023_2024.parquet']

for names,i in tqdm(df_names):
    print(f"Récupération depuis le cloud {i}/13")
    url = url_dataRNE[i-1]
    df = pd.read_parquet(url)
    dfs_rne[names] = df
#------------------------------------------------------------------------------------------------#

#Récupération des colonnes et création d'un nouveau data frame vide
columns_rne = list(dfs_rne[df_names[0][0]].columns)
idf_10_24 = pd.DataFrame(columns=columns_rne)

#Filtrage par activites et département
for names in tqdm(df_names):
    print("Filtrage par département et activités")
    data = dfs_rne[names[0]]
    data = data[data["codeAPE"].str.startswith(tuple(activites)) & data["code_postal"].str.startswith(tuple(dep))]
    idf_10_24 = pd.concat([idf_10_24,data])
    
#Filtrage pour les dates de création après 2010-01-01 (inlcus)
idf_10_24["date_creation"] = pd.to_datetime(idf_10_24["date_creation"])

print(f"Récupération depuis {date_min[0]}-{date_min[1]}-{date_min[2]}")
idf_10_24 = idf_10_24[idf_10_24["date_creation"] >= datetime.datetime(date_min[0],date_min[1],date_min[2])]

idf_10_24["dateRadiation"] = idf_10_24["dateRadiation"].apply(lambda x : 
    x if x not in ["non-disp","None"] else pd.NaT)


#------------------------------------------------------------------------------------------------#

#Récupération des colonnes et création d'un nouveau data frame vide
columns_rne = list(dfs_rne[df_names[0][0]].columns)
total = pd.DataFrame(columns=columns_rne)

#Récupération complète dans un data frame
print("Récupération de l'historique total")
for names in tqdm(df_names):
    data = dfs_rne[names[0]]
    total = pd.concat([total,data])


total["date_creation"] = pd.to_datetime(total["date_creation"])
total["dateRadiation"] = total["dateRadiation"].apply(lambda x : 
    x if x not in ["non-disp","None"] else pd.NaT)



  0%|          | 0/13 [00:00<?, ?it/s]

Récupération depuis le cloud 1/13


  8%|▊         | 1/13 [00:03<00:40,  3.38s/it]

Récupération depuis le cloud 2/13


  8%|▊         | 1/13 [00:05<01:02,  5.18s/it]


KeyboardInterrupt: 

In [ ]:
def get_RNE_local_concurrents(df_p : pd.DataFrame, total_p : pd.DataFrame) -> pd.DataFrame :

      total_p = total_p.loc[total_p["code_postal"]!="non-disp"]

      #Récupération de l'indicatrice 1 si radiée 0 sinon
      total_p["radié"] = total_p["dateRadiation"].apply(lambda x : 1 if x != None else 0)

      #Conversion en date time de date_creation par sécurité
      total_p["date_creation"] = pd.to_datetime(total_p["date_creation"])
      df_p["date_creation"] = pd.to_datetime(df_p["date_creation"])

      total_p["dateRadiation"] = total_p["dateRadiation"].apply(lambda x : 
      x if x not in ["non-disp","None"] else pd.NaT)
      total_p["dateRadiation"] = pd.to_datetime(total_p["dateRadiation"],errors="coerce")

      total_p["code_postal"] = total_p["code_postal"].astype(int)
      df_p["code_postal"] =  df_p["code_postal"].astype(int)     

      total = total_p.copy()
      df = df_p.copy()

      #Connexion DuckDB
      con = duckdb.connect()

      #Enregistrement des DataFrames pandas dans DuckDB
      con.register("df", df)        
      con.register("total", total)

      con.execute("""
      CREATE OR REPLACE VIEW total_long AS
      SELECT siren, date_creation,
            radié AS rad,
            codeAPE as ape,
            code_postal as code
      FROM total
      WHERE radié IS NULL;
      """)

      #Requête principale pour le nombre de concurrents encore en poste sur le marché local
      res = con.execute(f"""
      SELECT
      df.siren,

      COUNT(DISTINCT t.siren) AS nb_local_concurrents

      FROM df

      LEFT JOIN total_long t
      ON df.codeAPE = t.ape
      AND df.code_postal = t.code
      AND t.date_creation <= df.date_creation

      GROUP BY df.siren
      ORDER BY df.siren
      """).df()

      con.close()

      return res
    

In [36]:
df = idf_10_24
essai = get_RNE_local_concurrents(df,total)

ValueError: invalid literal for int() with base 10: 'non-disp'

In [ ]:
total.loc[total["code_postal"]=="non-disp"].shape